# Explaining Facial Expression Recognition with Simplification and Feature Attribution
# Notebook 1:  XAI for Affective Computing (SoSe2024)

In this notebook you will attempt to generate explanations for predictions of two Facial Expression recognition models, one for tabular (structured) data extracted from images and one for raw image data, both trained using a subset of the [AffectNet dataset](http://mohammadmahoor.com/affectnet/). AffectNet is an image dataset of facial expressions in the wild, and is labeled with 8 facial expression categories: **Neutral, Happy, Sad, Surprise, Fear, Disgust, Anger, and Contempt**. (Have a look at the paper for more details https://arxiv.org/abs/1708.03985). 

In **Part 1**, you will first explore local and global explanations on a tabular dataset of [Facial Action Units (FAUs)](https://imotions.com/blog/facial-action-coding-system/) by using the [LIME python package](https://github.com/marcotcr/lime). The dataset is comprised of FAUs that were extracted from the face images of AffectNet using [OpenFace2.0](https://github.com/TadasBaltrusaitis/OpenFace). This dataset is then used to train a XGBoost classifier (trained model is provided in the code below).

In **Part 2**, you will generate local explanations for a Convolutional Neural Network (CNN) using both LIME and a Backpropagation based method, either Layerwise Relevance Propagation or GradCAM.  The CNN is already trained using the raw images of AffectNet (trained model is provided in the code below). A subset of the test data is provided for generating and evaluating the explantions.  **_For this part you can choose between either TASK 3a, Saliency Maps with iNNvestigate, or TASK 3.6, GradCAM._**

To use this notebook, please make sure to go step by step through each of the cells review the code and comments along the way.

See **README** to get started with the Notebook setup

## Part 0: Notebook Setup

In [ ]:
%load_ext autoreload
%autoreload 2

**Import necessary libraries**

(see README for necessary package installations if you receive a `module not found` error.

In [ ]:
import pickle
from pathlib import Path

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["TF_USE_LEGACY_KERAS"] = "1"         # don't use keras 3

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

# import tensorflow for model loading
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# import sklearn for processing data and results
from sklearn.metrics import confusion_matrix, classification_report, auc, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

# import model loading function
from model import cnn_model, create_bn_replacment

# import plotting helper functions
import utils

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

## Part 1: Explanations of Facial Action Units

In this part, we will generate explanations for the XGBoost model trained using a dataset of Facial Action Units (as described in the notebook introduction).  

First, let's load the data and the trained models. Then we will evaluate the model peformance, before we start with the explanations.

### Load the data

In [ ]:
# Full data from training and evaluation
train_csv = '../data/affectnet_aus/train_aus.csv'
val_csv = '../data/affectnet_aus/val_aus.csv'

# load training and validation data as pandas dataframeas
df_train = pd.read_csv(train_csv)
df_val = pd.read_csv(val_csv)

# smaller dataset for explanations (same data as in Task 1)
xai_csv = '../data/affectnet_aus/eval_aus.csv'
df_xai = pd.read_csv(xai_csv)

# get only the columns storing action units from the dataframe
feature_names = [col for col in df_val if col.startswith('AU')]
categorical_features = [feat for feat in feature_names if '_c' in feat]
categorical_idxs = [i for i, feat in enumerate(feature_names) if '_c' in feat]

class_names = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']  # same class labels as before

In [ ]:
# convert data from dataframe to Numpy arrays
X_train = df_train.loc[:, feature_names]
y_train = df_train['class']
# set categorical features in dataframe for XGboost
for col in categorical_features:
    X_train[col] = X_train[col].astype(int).astype('category')

X_test = df_val.loc[:, feature_names]
y_test = df_val['class']
# set categorical features in dataframe for XGboost
for col in categorical_features:
    X_test[col] = X_test[col].astype(int).astype('category')

X_xai = df_xai.loc[:, feature_names]
y_xai = df_xai['class']
# set categorical features in dataframe for XGboost
for col in categorical_features:
    X_xai[col] = X_xai[col].astype(int).astype('category')

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)
print('XAI', X_xai.shape, y_xai.shape)

### Load pretrained XGBoost model
And validate that it works.  
The accuracy of the model in the training data should be around $92\%$

In [ ]:
from xgboost import XGBClassifier
random_state = 24

In [ ]:
# # Train model from scratch
# clf = XGBClassifier(booster='gbtree', enable_categorical=True, max_depth=15, eta=0.1, reg_lambda=30, random_state=random_state) 
# clf.fit(X_train, y_train)
# clf.score(X_train, y_train)
# clf.save_model("../models/xgboost_model.json")

In [ ]:
# Load pretrained model
clf = XGBClassifier(booster='gbtree', enable_categorical=True, max_depth=15, eta=0.1, reg_lambda=30, random_state=random_state)
clf.load_model("../models/xgboost_model.json")
clf.score(X_train, y_train)

### Now evaluate on the test data
Unfortunately, the accuracy is only $44\%$ but this is still well above chance guessing which would be $1 / 8 * 100 = 12.5\%$ accuracy (since there are 8 total classes)

In [ ]:
# get model predictions
y_test_preds = clf.predict(X_test)
y_test_true = y_test

# eval results
print(classification_report(y_test_true, y_test_preds, target_names=class_names))

We can also review the confusion matrix to see where the model makes its mistakes

In [ ]:
cm_data = confusion_matrix(y_test_true, y_test_preds)
cm = pd.DataFrame(cm_data, columns=class_names, index=class_names)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (20,10))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

### Evaluate with XAI Data
Here we calculate and evaluate predictions on a smaller `X_xai` dataset.  `X_xai`is a subset of the full test data (from above) and will be used throughout the rest of part 1.

The accuracy should be around $40\%$

In [ ]:
# get model predictions
y_xai_preds = clf.predict(X_xai)
y_xai_true = y_xai

print(classification_report(y_xai_true, y_xai_preds))

**Display confusion matrix for `X_xai` dataset**

In [ ]:
cm_data = confusion_matrix(y_xai_true, y_xai_preds)
cm = pd.DataFrame(cm_data, columns=class_names, index=class_names)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (20,10))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

### TASK 1: Implement LIME Local Explanations and SP-LIME for Global Explanations

Now on to the implementation LIME explanations. 

#### Task 1.0: 
**Identify a Few Images to Explain**

The code below will display images from the XAI dataset.
- Try changing value of `start` to get a new set of images (there are 10 images for each class; for example, the class happy will be at indexes 10-19)
- Search through the images to find at least 4 to explain 
    - Find classes that you would like to explain, and from each class select 2 images
        - one should be a correct prediction  
        - and one should be an incorrect prediction

In [ ]:
# packages needed for the rest of the tasks
from skimage import io
import lime.lime_tabular

In [ ]:
# Gets all images from folder
images = [io.imread(f) for f in df_xai.image]

# gets labels for ground truch and predictions
true_labels = [class_names[idx] for idx in y_xai_true]
pred_labels = [class_names[idx] for idx in y_xai_preds]

In [ ]:
# displays first 9 images in array
start = 70
utils.display_nine_images(images, true_labels, pred_labels, start)

In [ ]:
#### Enter the Indexes Here ### 
###############################
# you will use this array later in the task
img_idxs = []

#### Task 1.1
Now implement a [LimeTabularExplainer](https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.lime_tabular), you can review the [LIME tutorial](https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20continuous%20and%20categorical%20features.html) for help. Make sure to indicate which feature are categorical features as stored in the `categorical_features` variable, as continuous and categorical features are treated differently with LIME.

Note: In the feature names, you will see features with a `_c` and a `_r` at the end.  The `_r` means the intentsity of the action unit (i.e., how strong is it's presence), and the `_c` is a binary feature indicating the presence (value=1), or non-presence (value=0), of an action unit.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 1.2
Generate LIME explanations the previously identified 4 data instances from the `X_xai` dataset, using the `LimeTabularExplainer` and then plotting the explanations for each data instance (see tutorial mentioned above).  

HINT: Before showing an explanation, plot the image using `utils.display_one_image()` utility function.  Use `plt.show()` immediately after calling `utils.display_one_image()` to display the image before the explanation charts.

Make sure to print out the **True** and **Predicted** labels for each instance.

Try experimenting with different parameters for the explainer and explanation.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################



#### Talk 1.3
Identify the important Facial Actions Units and compare with the images at [Facial Action Units](https://imotions.com/blog/facial-action-coding-system/).  What insights do these local explanations provide?

Write your answer here...

- 

#### Task 1.4

Now implement [Submodular Pick](https://lime-ml.readthedocs.io/en/latest/lime.html#lime-submodular-pick-module) instance to generate global explanations and see if it provides you with a more global perspective of how the model makes decisions. You can review the [LIME tutorial](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Submodular%20Pick%20examples.ipynb) for help.

Try setting `num_exps_desired` to 16 to try to get 2 examples per class (although this isn't guaranteed).

In [ ]:
# import for submodular pick
from lime import submodular_pick

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 1.5
Now plot the explantions

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Bonus Task
Generate a pandas dataframe of the explanations and explore the dataframe to gain more insight into the explanations, see the [LIME tutorial](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Submodular%20Pick%20examples.ipynb) for an example.

One idea would be to aggregate the most important features for a given emotion and plot a global importance bar chart.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################



#### Task 1.6

How does LIME Submodular Pick select explanations for a global perspective on the model?

Identify important AUs for each of the classes and compare with the images at [Facial Action Units](https://imotions.com/blog/facial-action-coding-system/).  What insights do these explanations provide? Do you now have a better understanding of how the model is working? If not, what is lacking using the LIME approach and/or what could be done differently?

Write your answer here

...

## Part 2:  Local Explations of Facial Expression Recognition with Images

In this part, we will generate explanations for the CNN trained using facial images (as described in the notebook introduction).

First, let's load the data and the trained models. Then we will evaluate the model peformance, before we start with the explanations.

Set some global variables

In [ ]:
SEED = 12
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 80 
NUM_CLASSES = 8
CLASS_LABELS = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']

### Load Pretrained CNN Model and Setup Data Generator

In [ ]:
# make sure you've downloaded the models from Moodle (see README instructions for Notebook I)
model_path = '../models/affectnet_model_e=60/affectnet_model'

# test loading weights
model_xai = cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
model_xai.load_weights(model_path).expect_partial()

In [ ]:
test_dir = '../data/affectnet/val_class/'

# Load data
test_datagen = ImageDataGenerator(validation_split=0.2,
                                  rescale=1./255)
test_gen = test_datagen.flow_from_directory(directory=test_dir,
                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                            batch_size=BATCH_SIZE,
                                            shuffle=False,
                                            color_mode='rgb',
                                            class_mode='categorical', 
                                            seed = SEED)
images, classes = next(test_gen) # since batch size is set to 80, this will load the entire test dataset

### Evaluation and Predictions
Here we evaluate the loaded model to ensure it is working as expected.  You should get around $55\%$ accuracy. While this is not a perfect classifier is above random guessing which is $1 / 8 * 100 = 12.5$ accuracy

Then we load predictions to use throughout the notebook. 

The predictions results can then be viewed with a confusion matrix to see where the model is confused

In [ ]:
loss, acc = model_xai.evaluate(test_gen, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# get softmax predictions from model
preds = model_xai(images)

# convert predictions to integers
y_pred = np.argmax(preds, axis=-1)
y_true = np.argmax(classes, axis=-1)

# print detailed results
print(classification_report(y_true, y_pred, target_names=CLASS_LABELS))

In [ ]:
# we can also review the confusion matrix
cm_data = confusion_matrix(y_true, y_pred)
cm = pd.DataFrame(cm_data, columns=CLASS_LABELS, index = CLASS_LABELS)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (20,10))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

### TASK 2: LIME Local Prediction Explanations

Now that we have our model setup, we will review the images and predictions to identify a few data instances to explain.  

#### Task 2.0

Identify a Few Images to Explain

The code below will display images from the XAI dataset.

- Try changing start value to get a new set of images (there are 10 images for each class, so for example, the class happy will be at indexes 10-19)
- Search through the images to find at least 4 to explain 
    - Find classes that you would like to explain, and from each class select 2 images
        - one should be a correct prediction  
        - and one should be an incorrect prediction
        - These can be the same as previously select (but note that that the indexes are different due to different methods of reading the files from disk)

In [ ]:
# displays first 9 images in array
start = 0

true_labels = [CLASS_LABELS[idx] for idx in y_true]
pred_labels = [CLASS_LABELS[idx] for idx in y_pred]
utils.display_nine_images(images, true_labels, pred_labels, start)

In [ ]:
#### Enter the Indexes Here ### 
###############################
# you will use this array later in this task
img_idxs = []


#### Task 2.1 
**Implement a LIME Image Explainer**

Implement a [LimeImageExplainer](https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.lime_image) instance, you can review the [LIME tutorial](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb) for help. 

In [ ]:
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm

from skimage.segmentation import mark_boundaries # used to get boundries from explanation for plotting

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 2.2

As we discussed in the seminar, LIME requires images to be segmented into superpixels.  For facial expressions the segmentation algorithm is very important and the default of the `explain_instance()` method may not provide good explanations.  Experiment with different segmenters using the LIME `SegmentationAlgorithm` class and pass to `segmenter_fn` argument of the `explain_instance()` method.

- For example: `segmenter = SegmentationAlgorithm('method name', params)`, where params are defined by the skimage segmentation alorithm method
- by default LIME uses: 
```
segmenter = SegmentationAlgorithm('quickshift', kernel_size=4,
                                  max_dist=200, ratio=0.2,
                                  random_seed=random_seed)
```

You can find different algorithms via [skimage segmentation](https://scikit-image.org/docs/stable/api/skimage.segmentation.html)

For this task, implement and visualize the default LIME segmenter and at least one other segementation algorithm, and choose which you think is best for generating a LIME explanation.  

In [ ]:
##### YOUR CODE GOES HERE #####
###############################
# segmenter = SegmentationAlgorithm('quickshift', kernel_size=2,
#                                   max_dist=10, ratio=0.2,
#                                   random_seed=42)


#### Task 2.3

Now generate the explanations for each of the selected images. Experiment with the default segmenter and another segmenter as found above in task 2.2a



In [ ]:
##### YOUR CODE GOES HERE #####
###############################



#### Task 2.4: 

**Visualize Explanations**

Visualize the explanations for each of the 4 data points from LIME using matplotlib's `imshow` function (see above tutorial). (Or pass the explanation to the `display_one_image` from the `utils` module.)

*HINT*: Use the `subplot` parameter of the `display_one_image` to plot a 2x2 grid.  The value should be an integer formated as `RCN` where `R` is the number of rows, `C` is the number of columns, and `C` is the number of the image to plot.  For example, `221` means to plot the first image of a 2x2 grid, `222` means the plot the second images, and so forth... (also see `display_nine_images` for example of this usage.)

Experiment with at least 2 different sets of parameters for the explanation visualizations.  For example, view positive and negative contributions, change the number of features for the explation, or try visualizing a heatmap

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 2.5 
**Report on your findings**

What are your insights have you gained about the predictions? Can you identify any patterns that explain how the model is working? Are you more or less confident in the model's performance after reviewing the explanations?

Also, discuss how you might generate global explanations in this case.  Since SP-LIME doesn't work directly on images, can you think of other ways to generate global explanations from LIME?

Write here findings here...

### TASK 3a: Saliency Maps

In this task you will create and compare saliency maps using 3 different approaches from the tensorflow based package, [iNNvestigate](https://github.com/albermax/innvestigate). You can review the MNIST example notebook to see how the package works, https://github.com/albermax/innvestigate/blob/master/examples/mnist_compare_methods.ipynb

First we need to load and setup the model for use with the package.  Gradient based methods require that we remove the SoftMax layer from the network, and additionally we have to replace the Batch Normalization layers for methods other than Layerwise Relevance Propagation.  This can be done using the `prep_innvestigate_model` function below. For example, for using the model with LRP:

```python
# if using an innvestigate methods other than LRP set replace_bn=True
model_inn = prep_innvestigate_model(model, replace_bn=False)
```

In [ ]:
import innvestigate as inn
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
tf.compat.v1.disable_eager_execution() # we have to disable eager execution for iNNvestigate

In [ ]:
############################
# Post Processing Utilities
############################
def postprocess(X):
    X = X.copy()
    X = iutils.postprocess_images(X)
    return X


def bk_proj(X):
    return ivis.graymap(X)


def heatmap(X):
    # innvestigate heatmap doesn't appear to work well with batch of more than 1 image, 
    # so this replaces the default
    # Aggregate along color channels and normalize to [-1, 1]
    exps_ag = X.sum(axis=-1)
    exps_ag /= np.max(np.abs(exps_ag), axis=(-2, -1))[:, None, None]
    return exps_ag


def graymap(X):
    return ivis.graymap(np.abs(X), input_is_positive_only=True)

In [ ]:
def prep_innvestigate_model(model, replace_bn=False):
    """ Prepare model for use with innvestigate package
    
    Args:
        model: tensorflow model to be explained
        replace_bn (bool): should the batch normalization layers be replace with standard normalization. 
                           This is needed for most innvestigate methods except LRP
    Returns:
        tensorflow model that is compatible with innvestigate
    """
    model_tmp = model
    # innvestigate doesn't support batch norm for some models
    # so we much substitute BN layers with manual normalization layers
    # https://github.com/albermax/innvestigate/issues/292
    if replace_bn:
        bn_layers = [2, 6, 10, 14, 19, 22]
        input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
        layers = model.layers
        for idx in bn_layers:
            layers = create_bn_replacment(layers, idx)
        
        inputs = tf.keras.Input(shape=input_shape)
        x = inputs
        for l in layers:
            x = l(x)
        outputs = x
        model_tmp = tf.keras.Model(inputs=inputs, outputs=outputs)

    # remove softmax for gradient computations and return model
    return inn.model_wo_softmax(model_tmp)


In [ ]:

# reload model since we disabled eager execution
model = cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
model.load_weights(model_path).expect_partial()

#### Task 3a.1

In this task we will setup at least three different explainers, make sure one is an LRP implementation. 

- To do this first, setup a dictionary for your explainers and their parameters. Then create a list of analyzers initialized from the dictionary of methods.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 3a.2

Now generate the explanations for your four selected images

#### Task 3a.3

Use matplotlib to generate a grid of explanation similar to the MNIST example.  Each row should be an image to be explained, and each column is one of the specific methods.  Make sure to also print somewhere the image ground truth and model prediction. These should be stored in the variables `true_labels` and `pred_labels` from Task 2.0.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 3a.4

How do the methods compare? Do find that one is better than the other for explaining the predicition of the model?

Type your answer here...

### TASK 3b: Grad-CAM

We don't discuss GradCAM (Gradient-weighted Class Activation Maps) in the seminar this semester, but it is a commonly used approach for generating saliency maps. While it is a backpropagation based method, it takes a slightly different approach than other methods.  Instead, of backpropagating all the way to the input pixels, GradCAM calculates gradients only to final convolutional layers.  The gradient values (ie importance scores) are then used to linearly combined the activation maps from the last layer by weighting each map by the sum of the gradients for that map. [See the orignal paper for full details](https://arxiv.org/abs/1610.02391)

#### Task 3b.1
**Implement the GradCAM Algorithm**

For this task  you will implement a version of GradCAM based on the [Keras GradCAM Tutorial](https://keras.io/examples/vision/grad_cam/). To better understand the algorithm, rather than just copying and pasting the entire functions, try implementing each main step of the alorithm in a seperate cell.  Then review the output of that cell either by printing the tensor or tensor shape.  Or by visualizing the output with matplotlib, for example with `plt.imshow()`

**Note:**  the final convolution layer in our network is named `final_conv_layer`

**Note 2:** if you've also completed Task 3a, you will need to restart your notebook. Then rerun without enabling eagar execution in the import cell of Task 3a 

In [ ]:
# reload model
model_gcam = cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
model_gcam.load_weights(model_path).expect_partial()

In [ ]:
##### YOUR CODE GOES HERE #####
###############################



#### Task 3b.2

Wrap the GradCAM Implementation into functions for generating the heatmaps and creating a superimposed image.  Instead of saving the superimposed image to a file (as done in the tutorial), simply generate the superimposed image and return it. We will use it in the next task.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 3b.3

Using your new GradCAM functions and the `display_one_image()` function in the `utils` model display the heatmaps for your images.  However, instead of just displaying the heatmaps for the top predicated class, for each of your images generate a heat map for each class label, see the file `gradcam_example.png` for an example of what this might look like. Also, for each image, add a border to the image representing the predicted class.  See the docstring of `display_one_image()` for details on usage.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################

##### YOUR CODE GOES HERE #####
###############################


#### Task 3b.5

What patterns do you notice in the heatmaps for the different classes. Do the regions of the image where the model focuses make sense? Or do you notice any unusual biases?

### TASK 4: Final Discussion
Between tasks 1, 2, and 3, which of the 2 methods best help you understand how the model is function, as we've discussed throughout the seminar. Why?

write your answer here...

### BONUS TASK

Try plot LIME output and GradCAM outpt for the same images next too each other.  Are the results similar or are there difference in indentified important regions?

Or feel free to generate your own plots or visualizations to help understand the explanations and models better.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################

